In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sklearn.preprocessing as preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/its-a-fraud/train.csv")

In [ ]:
test=pd.read_csv("../input/its-a-fraud/test.csv")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
train.describe()

In [ ]:
train.dtypes

In [ ]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

missing(train)

In [ ]:
limitPer = len(train) * .50
train.dropna(thresh=limitPer, axis=1,inplace=True)

In [ ]:
train.shape

In [ ]:
missing(train)

In [ ]:
train.duplicated().sum()

In [ ]:
train.dtypes

In [ ]:
n = train.nunique(axis=0)
n

In [ ]:
wt=train["isFraud"].value_counts(normalize=True).to_frame()
wt.plot.bar()
wt.T

In [ ]:

cat_features = ['ProductCD', 'card1','card2','card3','card4','card5','card6', 'addr1','addr2', 'P_emaildomain',
                'M1', 'M2', 'M3', 'M4', 'M6']

num_features = [x for x in train.columns.values[2:] if x not in cat_features]

In [ ]:
print('Categorical features :', len(cat_features))
print('Numerical features : ',len(num_features))

In [ ]:
plt.figure(figsize=(12,6))
train_ProductCD = (train.groupby(['isFraud'])['ProductCD']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('ProductCD'))
sns.barplot(x="ProductCD", y="percentage", hue="isFraud", data=train_ProductCD, palette=["#FFD500", "#005BBB"]);

In [ ]:
train[cat_features].head()

In [ ]:
train[cat_features]=train[cat_features].fillna(train.mode().iloc[0])


In [ ]:
train[cat_features].head()

In [ ]:
train[num_features].head()

In [ ]:
train[num_features]=train[num_features].fillna(train.median())

In [ ]:
train[num_features].head()

In [ ]:
train.isnull().sum().sum()

In [ ]:
train[num_features].describe()

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar
def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
   
    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts


In [ ]:
plt.figure(figsize=(15,18))
plt.subplot(3,3,1)
train_m1 = (train.groupby(['isFraud'])['M1']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M1'))
sns.barplot(x="M1", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_m1)
plt.title('TrainM1')
plt.subplot(3,3,2)
train_m2 = (train.groupby(['isFraud'])['M2']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M2'))
sns.barplot(x="M2", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_m2)
plt.title('TrainM2')
plt.subplot(3,3,3)
train_m3 = (train.groupby(['isFraud'])['M3']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M3'))
sns.barplot(x="M3", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_m3)
plt.title('TrainM3')
plt.subplot(3,3,4)
train_m4 = (train.groupby(['isFraud'])['M4']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M4'))
sns.barplot(x="M4", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_m4)
plt.title('TrainM4')
plt.subplot(3,3,5)
train_m6 = (train.groupby(['isFraud'])['M6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('M6'))
sns.barplot(x="M6", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_m6)
plt.title('TrainM6')

In [ ]:
train["M4"].unique()

In [ ]:
m_cols = [c for c in train if c[0] == 'M']
train[m_cols].head()

In [ ]:
m_col = train[['M1','M2','M3','M4','M6']]

In [ ]:
#for removing columns with only one unique value if any
for col in train.columns:
    if len(train[col].unique()) == 1:
        train.drop(col,inplace=True,axis=1)

In [ ]:
(train[m_cols] == 'T').sum().plot(kind='bar', color='#87CEFA', title='Count of T by M column', figsize=(15, 2))
plt.show()
(train[m_cols] == 'F').sum().plot(kind='bar', color='#87CEFA', title='Count of F by M column',figsize=(15, 2))
plt.show()
(train[m_cols].isna()).sum().plot(kind='bar', color='#87CEFA', title='Count of NaN by M column',figsize=(15, 2))
plt.show()

In [ ]:
train.groupby('M4')['TransactionID'].count().plot(kind='bar',
                                                  color='#87CEFA',
                                                  title='Count of values for M4',
                                                  figsize=(15, 3))
plt.show()

In [ ]:
C_cols = [c for c in train if c[0] == 'C']
train[C_cols].head()


In [ ]:
cor_c = train[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','isFraud']]

In [ ]:
corr_matrix = cor_c.corr()
plt.figure(figsize=(15, 8))

im, _ = heatmap(corr_matrix, cor_c, cor_c, 
                cmap="YlGn", vmin=-1, vmax=1,
                cbarlabel="correlation coeff.")


def func(x, pos):
    return "{:.2f}".format(x).replace("0.", ".").replace("1.00", "")

annotate_heatmap(im, valfmt=matplotlib.ticker.FuncFormatter(func), size=10)


plt.tight_layout()
plt.show()




In [ ]:
d_cols = [c for c in train if c[0] == 'D']
train[d_cols].head()

In [ ]:
plt.figure(figsize=(15,18))
plt.subplot(3,3,1)
train_d1 = (train.groupby(['isFraud'])['D1']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(.100)
                     .reset_index()
                     .sort_values('D1'))
sns.barplot(x="D1", y="percentage", hue="isFraud", palette=['#90EE90', '#FA8072'], data=train_d1)
plt.title('TrainD1')



In [ ]:
def describe(datatrain,datatest,feature):
    d = pd.DataFrame(columns=[feature,'Train','TrainFraud','TrainLegit','Test'])
    d[feature] = ['count','mean','std','min','25%','50%','75%','max','unique','NaN','NaNshare']
    for i in range(0,8):
        d['Train'].iloc[i] = datatrain[feature].describe().iloc[i]
        d['TrainFraud'].iloc[i]=datatrain[datatrain['isFraud']==1][feature].describe().iloc[i]
        d['TrainLegit'].iloc[i]=datatrain[datatrain['isFraud']==0][feature].describe().iloc[i]
        d['Test'].iloc[i]=datatest[feature].describe().iloc[i]
    d['Train'].iloc[8] = len(datatrain[feature].unique())
    d['TrainFraud'].iloc[8]=len(datatrain[datatrain['isFraud']==1][feature].unique())
    d['TrainLegit'].iloc[8]=len(datatrain[datatrain['isFraud']==0][feature].unique())
    d['Test'].iloc[8]=len(datatest[feature].unique())
    d['Train'].iloc[9] = datatrain[feature].isnull().sum()
    d['TrainFraud'].iloc[9] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()
    d['TrainLegit'].iloc[9] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()
    d['Test'].iloc[9]=datatest[feature].isnull().sum()
    d['Train'].iloc[10] = datatrain[feature].isnull().sum()/len(datatrain)
    d['TrainFraud'].iloc[10] = datatrain[datatrain['isFraud']==1][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==1])
    d['TrainLegit'].iloc[10] = datatrain[datatrain['isFraud']==0][feature].isnull().sum()/len(datatrain[datatrain['isFraud']==0])
    d['Test'].iloc[10]=datatest[feature].isnull().sum()/len(datatest)
    return d

In [ ]:
transactionAmtDescribe = describe(train,test,'TransactionAmt')

In [ ]:
transactionAmtDescribe

In [ ]:
l=[99.9,99.91,99.92,99.93,99.94,99.95,99.96,99.97,99.98,99.99]
for i in l:
    print('train',np.percentile(train['TransactionAmt'],i))
    print('test',np.percentile(test['TransactionAmt'],i))

In [ ]:
train['LogTransactionAmt'] = np.log(train['TransactionAmt'])
test['LogTransactionAmt'] = np.log(test['TransactionAmt'])

In [ ]:
plt.figure(figsize=(9,6))
plt.subplot(1,2,1)
sns.distplot(train[train['isFraud']==0]['LogTransactionAmt'])
sns.distplot(train[train['isFraud']==1]['LogTransactionAmt'])
plt.legend(['legit','fraud'])
plt.title('Train')
plt.subplot(1,2,2)
sns.distplot(test['LogTransactionAmt'])

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card4 = (train[~train['card4'].isnull()].groupby(['isFraud'])['card4']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card4'))
sns.barplot(x="card4", y="percentage", hue="isFraud", data=train_card4)
plt.title('Train')
plt.subplot(1,2,2)
test_card4 =test[~test['card4'].isnull()]['card4'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card4)
plt.title('Test')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card6 = (train[~train['card6'].isnull()].groupby(['isFraud'])['card6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card6'))
sns.barplot(x="card6", y="percentage", hue="isFraud", data=train_card6)
plt.title('Train')
plt.subplot(1,2,2)
test_card6 =test[~test['card6'].isnull()]['card6'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card6)
plt.title('Test')


In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
train_card4 = (train[~train['card4'].isnull()].groupby(['isFraud'])['card4']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card4'))
sns.barplot(x="card4", y="percentage", hue="isFraud", data=train_card4)
plt.title('Train')
plt.subplot(1,2,2)
test_card4 =test[~test['card4'].isnull()]['card4'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card4)
plt.title('Test')

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
train_card6 = (train[~train['card6'].isnull()].groupby(['isFraud'])['card6']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('card6'))
sns.barplot(x="card6", y="percentage", hue="isFraud", data=train_card6)
plt.title('Train')
plt.subplot(1,2,2)
test_card6 =test[~test['card6'].isnull()]['card6'].value_counts(normalize=True).mul(100).rename('percentage')\
.reset_index()
sns.barplot(x="index", y="percentage", data=test_card6)
plt.title('Test')

In [ ]:
train['P_emaildomain'].value_counts()[:10]

In [ ]:
def returnfirst(email):
    return email.split(".")[0]

In [ ]:
train['first'] = train[~train['P_emaildomain'].isnull()]['P_emaildomain'].apply(returnfirst)

In [ ]:
test['first'] = test[~test['P_emaildomain'].isnull()]['P_emaildomain'].apply(returnfirst)

In [ ]:
train_email = (train.groupby(['isFraud'])['first']
                     .value_counts(normalize=True)
                     .rename('percentage')
                     .mul(100)
                     .reset_index()
                     .sort_values('first'))

In [ ]:
plt.figure(figsize=(22,10))
sns.barplot(x="first", y="percentage", hue="isFraud", data=train_email)
plt.xticks(rotation=90)

In [ ]:
D_cols = [c for c in train if c[0] == 'D']
train[D_cols].head()

In [ ]:
D_c = train[['D1','D2','D3','D4','D10','D11','D15','isFraud']];
f = D_c.corr()

In [ ]:
plt.figure(1,figsize=(12,12))
sns.heatmap(f,annot=True)
plt.title('D train')

In [ ]:
train["M4"].unique()

In [ ]:
train["P_emaildomain"].unique()

In [ ]:
colsToHotEncode=["ProductCD","card4","card6","M4"]

In [ ]:
train=pd.get_dummies(train,columns=colsToHotEncode)

In [ ]:
train["M1"].replace(['T', 'F'], [1, 0], inplace=True)
train["M2"].replace(['T', 'F'], [1, 0], inplace=True)
train["M3"].replace(['T', 'F'], [1, 0], inplace=True)
train["M6"].replace(['T', 'F'], [1, 0], inplace=True)

In [ ]:
frq=train.groupby("P_emaildomain").size()
frq

In [ ]:
frq_dist=frq/len(train)
frq_dist

In [ ]:
train["P_emailDomainFreq"] = train.P_emaildomain.map(frq_dist)

In [ ]:
train.drop("P_emaildomain",axis=1,inplace=True)

In [ ]:
#for removing columns with only one unique value if any
for col in train.columns:
    if len(train[col].unique()) == 1:
        train.drop(col,inplace=True,axis=1)


In [ ]:
class OutlierRemoval: 
    def __init__(self, lower_quartile, upper_quartile):
        self.lower_whisker = lower_quartile - 1.5*(upper_quartile - lower_quartile)
        self.upper_whisker = upper_quartile + 1.5*(upper_quartile - lower_quartile)
    def removeOutlier(self, x):
        return (x if x <= self.upper_whisker and x >= self.lower_whisker else (self.lower_whisker if x < self.lower_whisker else (self.upper_whisker)))

In [ ]:
for i in num_features:
    num_var_outlier_remover = OutlierRemoval(train[i].quantile(0.25), train[i].quantile(0.75))
    train[i] = train[i].apply(num_var_outlier_remover.removeOutlier)

In [ ]:
train.describe() #removed outliers

In [ ]:
train.head()

In [ ]:
train.corr().isFraud

In [ ]:
ytrain=train["isFraud"]
xtrain=train
xtrain.drop("isFraud",inplace=True,axis=1)


In [ ]:
xtrain.drop("first",inplace=True,axis=1)

In [ ]:
#Applying PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(xtrain)


In [ ]:
candidate_components=np.arange(1,100,5)
variances=[]
for candidate in candidate_components:
    pca = PCA(n_components=candidate)
    principalComponents = pca.fit_transform(X_scaled)
    variances.append(np.sum(pca.explained_variance_ratio_))
    
plt.figure(figsize = (5, 5))
plt.plot(candidate_components,variances)
plt.xlabel('Number of components')
plt.ylabel('Variance')
plt.title('There will usually be an elbow in the curve, where the explained variance stops growing fast',fontsize=7);

In [ ]:
pca = PCA(n_components=40)
principalComponents = pca.fit_transform(X_scaled)
print(pca.explained_variance_ratio_)

In [ ]:
lr1 = LogisticRegression()
lr1.fit(principalComponents, ytrain)

In [ ]:
yhat_train_pca = lr1.predict(principalComponents)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(ytrain, yhat_train_pca)